#Research

In [3]:
import pandas as pd
import numpy as np

In [5]:
#Загузка datasets
df_train = pd.read_csv("/content/train_tin.csv", encoding='cp1251')
df_test = pd.read_csv("/content/test_tin.csv", encoding='cp1251')
df_train["type"]="train"
df_test["type"]="test"

In [6]:
df_train["text"].sample(1).iloc[0]

'Добрый день!При покупке через МБ Тинькофф билетов в кино должен начисляться дополнительный кэшбек в 15% (50% во время какой-то там акции), соответственно неоднократно покупал билеты, и НИ РАЗУ кэшбек не был начислен самостоятельно, только случайно обноружив однажды я написал, после чего начал проверять. и такой кейс повторялся трижды.\xa0После обращения в чат возврат делали, однако каждый раз контролировать и писать не очень хочется, как-то не юзер-френдли это.\xa0В КЦ (чат) обращался неоднократно, дельного ответа когда исправят так и не получил, поэтому пишу\xa0отзыв тут, т.к. знаю, что это поможет вам ускорить исправление данной проблемыБольшое спасибо, с уважением Артем М.'

In [7]:
df = pd.concat([df_train, df_test])

In [8]:
#Классы уравновешены
df_train['isPositive'].value_counts(normalize = True)

1    0.5
0    0.5
Name: isPositive, dtype: float64

#Предобработка текста

In [9]:
#Работа со стоп-словами
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
#Добавим часто упоминающееся
from nltk.corpus import stopwords
stop = stopwords.words("russian") + ["тинькофф"]

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
#Приведем текст к нижнему регистру
df["whithout"] = df['text'].apply(str.lower)

In [14]:
#Удалим стоп-слова из текстов
from nltk.tokenize import word_tokenize

def delete_stop(text):
  text_tokens = word_tokenize(text)
  tokens_without_sw = [word for word in text_tokens if not word in stop]
  return " ".join(tokens_without_sw)

df["whithout"] = df['whithout'].apply(delete_stop)

In [15]:
df['whithout']

0      27.09.19 сайт разделе `` рефинансирование стор...
1      добрый день ! неоднократно поступают звонки до...
2      первый решила воспользоваться кредитной картой...
3      самом дело накипело из-за участившегося намаха...
4      дорожите своими нервами - каких обстоятельства...
                             ...                        
995    приветствую ! 18.02 хотел совершить несколько ...
996    сотрудник шахноза ( 6904552 ) очень грамотно б...
997    добрый день ! являюсь клиентом банка , 5 проду...
998    столкнулись проблемой : нужно провести платёж ...
999    всем здраствуйте , хочу оставить отзыв банке ,...
Name: whithout, Length: 4950, dtype: object

In [16]:
#Используя регулярные выражения, поработаем с удалением незначимых для модели токенов
import re
def regulars(sentance):
    sentance = re.sub(r'https?://\S+|www\.\S+', r'', sentance) # remove URLS
    sentance = re.sub(r'<.*?>', r'', sentance) # remove HTML
    sentance = re.sub(r' банк\w*', r'', sentance)
    sentance = re.sub(r'\d+', '', sentance).strip() # remove number
    sentance = re.sub(r"[^\w\s\d]","", sentance) # remove pnctuations
    sentance = re.sub(r'@\w+','', sentance) # remove mentions
    sentance = re.sub(r'#\w+','', sentance) # remove hash
    sentance = re.sub(r"\s+"," ", sentance).strip() # remove space
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    return sentance

In [18]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 8.2 MB 13.1 MB/s 


In [19]:
# Лемматизиуем текст, то есть привидем слова в начальную форму
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def parse(text):
    text_tokens = word_tokenize(text)
    for i in range(len(text_tokens)):
      text_tokens[i] = morph.parse(text_tokens[i])[0].normal_form
    return " ".join(text_tokens)

In [20]:
df["whithout"] = df['whithout'].apply(regulars)
df['whithout']

0      сайт разделе рефинансирование сторонних кредит...
1      добрый день неоднократно поступают звонки долж...
2      первый решила воспользоваться кредитной картой...
3      самом дело накипело изза участившегося намахал...
4      дорожите своими нервами каких обстоятельствах ...
                             ...                        
995    приветствую хотел совершить несколько покупок ...
996    сотрудник шахноза очень грамотно быстро помогл...
997    добрый день являюсь клиентом продуктам ранее в...
998    столкнулись проблемой нужно провести платёж фи...
999    всем здраствуйте хочу оставить отзыв прошел го...
Name: whithout, Length: 4950, dtype: object

In [21]:
df["whithout"] = df['whithout'].apply(parse)
df['whithout']

0      сайт раздел рефинансирование сторонний кредит ...
1      добрый день неоднократно поступать звонок долж...
2      первый решить воспользоваться кредитный карта ...
3      сам дело накипеть изз участиться намахалов сто...
4      дорожить свой нерв какой обстоятельство иметь ...
                             ...                        
995    приветствовать хотеть совершить несколько поку...
996    сотрудник шахноза очень грамотно быстро помочь...
997    добрый день являться клиент продукт ранее всё ...
998    столкнуться проблема нужно провести платёж физ...
999    весь здраствовать хотеть оставить отзыв пройти...
Name: whithout, Length: 4950, dtype: object

In [22]:
df['whithout'].sample(1).iloc[0]

'взять данный кредит тысяча рубль тысяча рубль тысяча рубль тысяча рубль отображаться мобильный приложение брать работать способный выплачивать кредит ситуация поменяться остаться работа соответственно смочь выплачивать данный сделать договор реструктуризация который вступать сила момент пополнение рубль мобильный приложение исчезнуть вообще сумма задолженность суть верить мобильный приложение должный везде нуль писать чат сотрудник показывать сумма долг отвечать это сделать деньга должный вносить терминал номер договор реструктуризация причём терминал тинькоффбанк чек выдавать предложить узнавать деньга поступать сам сотрудник'

# Classification task

Для получения представления текстов в числовом векторном пространстве, воспользуемся предобученной моделью типа трансформер, а точнее дистиллированной предобученной версией BERT для русского языка с библиотеки HuggingFace

In [23]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 6.9 MB/s 
     |████████████████████████████████| 636 kB 63.8 MB/s 
     |████████████████████████████████| 895 kB 74.1 MB/s 
     |████████████████████████████████| 3.3 MB 58.2 MB/s 
     |████████████████████████████████| 52 kB 2.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [24]:
from transformers import AutoTokenizer, AutoModel, AutoModelForPreTraining
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.5M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
#Вектором, описывающим текст, будет являться последний выходной слой модели токена CLS, который предназначен для задачи классификации
import torch
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy().T

In [26]:
#Создадим признак с эмбедингами текстов
df['embedings'] = df['whithout'].apply(embed_bert_cls, model = model, tokenizer = tokenizer)

In [27]:
#Создадим матрицу признаков
x = np.stack(df[df["type"]=='train']['embedings'].values)
y = df[df["type"]=='train']['isPositive'].values
x.shape

(3950, 312)

In [28]:
#Поделим dataset на тренировочную и тестовую выборку
from sklearn.model_selection import train_test_split
random_state = 13
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=random_state )

In [39]:
#В качестве алгоритма классификации выберем градиентный бустинг
from lightgbm import LGBMClassifier

lgmb = LGBMClassifier(
    learning_rate=0.03,
    num_leaves=20,
    max_depth=30, 
    objective="binary",
    n_estimators=500,
    n_jobs=-1
)

lgmb.fit(
    x_train,
    y_train,
    eval_set=(x_test, y_test),
    eval_metric="logloss",
    verbose=20
)

[20]	valid_0's binary_logloss: 0.505098	valid_0's binary_logloss: 0.505098
[40]	valid_0's binary_logloss: 0.41616	valid_0's binary_logloss: 0.41616
[60]	valid_0's binary_logloss: 0.368187	valid_0's binary_logloss: 0.368187
[80]	valid_0's binary_logloss: 0.339473	valid_0's binary_logloss: 0.339473
[100]	valid_0's binary_logloss: 0.321242	valid_0's binary_logloss: 0.321242
[120]	valid_0's binary_logloss: 0.309179	valid_0's binary_logloss: 0.309179
[140]	valid_0's binary_logloss: 0.30082	valid_0's binary_logloss: 0.30082
[160]	valid_0's binary_logloss: 0.294464	valid_0's binary_logloss: 0.294464
[180]	valid_0's binary_logloss: 0.290627	valid_0's binary_logloss: 0.290627
[200]	valid_0's binary_logloss: 0.287206	valid_0's binary_logloss: 0.287206
[220]	valid_0's binary_logloss: 0.284653	valid_0's binary_logloss: 0.284653
[240]	valid_0's binary_logloss: 0.283123	valid_0's binary_logloss: 0.283123
[260]	valid_0's binary_logloss: 0.281775	valid_0's binary_logloss: 0.281775
[280]	valid_0's bina

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.03, max_depth=30,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=-1, num_leaves=20, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [40]:
#Метрика качества roc_auc
#На выходе модели получаем вероятности принадлежности к классу 1 (положительный отзыв). При округлении получаем метку класса 
from sklearn.metrics import roc_auc_score
lgb_preds = lgmb.predict(x_test)
lgb_preds=lgb_preds.round(0)
lgb_preds=lgb_preds.astype(int)
roc_auc_score(lgb_preds,y_test)

0.8820391898098731

LightGBM fine-tune

In [ ]:
#Подбор гипер-параметров для повышения качества модели
#Зададим диапозон изменения параметров
learning_rates=np.arange(0.01, 0.06, 0.01)
num_leaves=list(range(2, 31, 6))
boosting_types=['gbdt', 'dart'] 
n_estimators=list(range(200, 801, 200))
max_depth=list(range(10, 51, 25))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([('classifier', LGBMClassifier(objective="binary", n_jobs=-1))])

param_grid = [
    {
    'classifier__learning_rate' : learning_rates,
    'classifier__num_leaves' : num_leaves,
    'classifier__boosting_types' : boosting_types,
    'classifier__n_estimators' : n_estimators,
    'classifier__max_depth' : max_depth,
    }    
]

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1, scoring='roc_auc')
clf.fit(x_train, y_train, classifier__eval_set=(x_test, y_test), classifier__eval_metric="logloss", classifier__verbose=100)
clf.best_score_

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 43.8min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 103.5min


In [1]:
clf.best_params_

NameError: ignored

In [41]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression(max_iter=1000)
logisticRegr.fit(x_train, y_train)
y_pred=logisticRegr.predict(x_test)
y_pred=y_pred.round(0)
roc_auc_score(y_pred,y_test)

0.8831682073366683

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

solvers=['lbfgs', 'sag', 'saga','newton-cg']
penalties=['l1', 'l2', 'elasticnet']
C = [0.001, 0.01, 0.1, 1, 10, 100]
pipe = Pipeline([('classifier', LogisticRegression(max_iter=1000))])

param_grid = [
    {
    'classifier__penalty' : penalties,
    'classifier__C' : C,
    'classifier__solver' : solvers
    }    
]

# Create grid search object

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(x_train, y_train)
best_clf.best_score_

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  1.7min finished


0.883860035451855

In [ ]:
best_clf.best_params_

{'classifier__C': 100,
 'classifier__penalty': 'l2',
 'classifier__solver': 'lbfgs'}

Неудалось потюнить lightgbm из отстутсвия видеокарты. Логрегрессия дает науилучшее значение метрики, будем использовать ее.

In [65]:
logRegr = LogisticRegression(C=100, max_iter=1000)
logRegr.fit(np.vstack([x_train, x_test]), np.concatenate([y_train,y_test]))
y_pred = logRegr.predict(np.stack(df[df["type"]=='test']['embedings'].values))

In [84]:
y_pred = y_pred.astype('int64')

In [86]:
df_pred = df[df['type'] == "test"][['text','isPositive']]

In [87]:
df_pred['isPositive']=y_pred

In [88]:
df_pred

,text,isPositive
0,Добрый день! Я являюсь клиентом Тинькофф банк ...,0
1,Хочу выразить огромную благодарность банку Тин...,1
2,Выражаю благодарность К-ву Александру В. за ст...,1
3,В январе 2019 года оформила потребительский кр...,0
4,Добрый день. Хочу поблагодарить банк Тинькофф ...,1
...,...,...
995,Приветствую! 18.02 хотел совершить несколько п...,0
996,Сотрудник Шахноза (6904552) очень грамотно и б...,1
997,"Добрый день!Я являюсь клиентом банка, теперь у...",0
998,Столкнулись с проблемой: нужно было провести п...,1
